## Earnings Surprise Prediction

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
## To check
import sovai as sov
sov.token_auth(token="your_token_here")

### Earnings Surprise

In [12]:
df_earn_surp = sov.data("earnings/surprise", tickers=["AAPL", "MSFT"])

In [13]:
df_earn_surp

surprise_probability  eps_surprise  actual_earning_result  \
ticker date                                                                    
AAPL   2016-12-30                -0.437         0.040                  0.840   
       2017-01-06                -0.443         0.040                  0.840   
       2017-01-13                -0.455         0.040                  0.840   
       2017-01-20                -0.470         0.040                  0.840   
       2017-01-27                -0.406         0.040                  0.840   
...                                 ...           ...                    ...   
MSFT   2024-06-14                -0.235           NaN                    NaN   
       2024-06-21                -0.234           NaN                    NaN   
       2024-06-28                -0.229           NaN                    NaN   
       2024-07-05                -0.211           NaN                    NaN   
       2024-07-12                -0.260           NaN                    NaN   

                   estimated_earning             date_pub  
ticker date                                                
AAPL   2016-12-30              0.800  2017-01-31T00:00:00  
       2017-01-06              0.800  2017-01-31T00:00:00  
       2017-01-13              0.800  2017-01-31T00:00:00  
       2017-01-20              0.800  2017-01-31T00:00:00  
       2017-01-27              0.800  2017-01-31T00:00:00  
...                              ...                  ...  
MSFT   2024-06-14              2.930                 None  
       2024-06-21              2.930                 None  
       2024-06-28              2.930                 None  
       2024-07-05              2.930                 None  
       2024-07-12              2.930                 None  

[788 rows x 5 columns]

**All Data:** This is a very large file (2 mins wait)

In [14]:
df_earn = sov.data("earnings/surprise", verbose=True)

/earnings/surprise
Requesting URL: https://data.sov.ai/earnings/surprise with params: {'parquet': 'True'}
Returning cached data


In [16]:
df_earn

surprise_probability  eps_surprise  actual_earning_result  \
ticker date                                                                    
A      2016-12-30                 0.190         0.040                  0.530   
       2017-01-06                 0.197         0.040                  0.530   
       2017-01-13                 0.210         0.040                  0.530   
       2017-01-20                 0.222         0.040                  0.530   
       2017-01-27                 0.236         0.040                  0.530   
...                                 ...           ...                    ...   
ZYXI   2024-06-21                -0.429           NaN                    NaN   
       2024-06-28                -0.403           NaN                    NaN   
       2024-07-05                -0.417           NaN                    NaN   
       2024-07-12                -0.413           NaN                    NaN   
       2024-07-19                -0.383           NaN                    NaN   

                   estimated_earning   date_pub  
ticker date                                      
A      2016-12-30              0.490 2017-02-14  
       2017-01-06              0.490 2017-02-14  
       2017-01-13              0.490 2017-02-14  
       2017-01-20              0.490 2017-02-14  
       2017-01-27              0.490 2017-02-14  
...                              ...        ...  
ZYXI   2024-06-21              0.080        NaT  
       2024-06-28              0.080        NaT  
       2024-07-05              0.080        NaT  
       2024-07-12              0.080        NaT  
       2024-07-19              0.080        NaT  

[1709948 rows x 5 columns]

In [33]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from sovai import data

# Load the earnings data
df_earn = data("earnings/surprise")

app = dash.Dash(__name__)

def calculate_slope(data):
    x = np.arange(len(data))
    slope, _ = np.polyfit(x, data, 1)
    return slope

def create_figure(df_company, df_price, ticker):
    df_company['rolling_slope'] = df_company['surprise_probability'].rolling(window=13).apply(calculate_slope)
    min_slope = df_company['rolling_slope'].min()
    max_slope = df_company['rolling_slope'].max()
    df_company['scaled_slope'] = np.interp(df_company['rolling_slope'], (min_slope, max_slope), (-0.5, 0.5))

    fig = go.Figure()

    # Earnings data
    fig.add_trace(go.Scatter(x=df_company.index, y=df_company['surprise_probability'], name='Predicted Surprise Probability', line=dict(color='#00FFFF', width=2)))
    fig.add_trace(go.Scatter(x=df_company.index, y=df_company['eps_surprise'], name='Real EPS Surprise', line=dict(color='#FF69B4', width=2)))
    fig.add_trace(go.Scatter(x=df_company.index, y=df_company['scaled_slope'], name='Surprise Probability Rolling Slope (13-period, Scaled)', line=dict(color='#d62728', width=1)))

    # Stock price data
    fig.add_trace(go.Scatter(x=df_price['date'], y=df_price['closeadj'], name='Stock Price', yaxis='y2', line=dict(color='#90EE90', width=2)))

    earnings_range = max(
        abs(df_company['surprise_probability'].min()),
        abs(df_company['surprise_probability'].max()),
        abs(df_company['eps_surprise'].min()),
        abs(df_company['eps_surprise'].max())
    )

    fig.update_layout(
        title=dict(text=f'{ticker}: Earnings Surprise and Stock Price', font=dict(size=20, color='white')),
        xaxis=dict(
            title='Date', 
            gridcolor='#2c2c2c', 
            showgrid=True, 
            linecolor='white', 
            tickfont=dict(color='white'),
            range=[df_company.index.min(), df_company.index.max()]  # Set x-axis range based on earnings data
        ),
        yaxis=dict(
            title='Earnings Metrics',
            range=[-earnings_range, earnings_range],
            gridcolor='#2c2c2c',
            showgrid=True,
            linecolor='white',
            tickfont=dict(color='white')
        ),
        yaxis2=dict(
            title='Stock Price',
            overlaying='y',
            side='right',
            gridcolor='#2c2c2c',
            showgrid=False,
            linecolor='white',
            tickfont=dict(color='white')
        ),
        plot_bgcolor='#1c1c1c',
        paper_bgcolor='#1c1c1c',
        legend=dict(
            x=0.5,
            y=-0.2,
            orientation='h',
            xanchor='center',
            font=dict(color='white'),
            bgcolor='#1c1c1c',
            bordercolor='white',
            borderwidth=1
        ),
        hovermode='x unified',
        hoverlabel=dict(font=dict(color='white'), bgcolor='#1c1c1c')
    )

    return fig

app.layout = html.Div([
    dcc.Dropdown(
        id='ticker-dropdown',
        options=[{'label': ticker, 'value': ticker} for ticker in df_earn.index.get_level_values('ticker').unique()],
        value='ADBE',  # Default value
        style={'width': '50%', 'margin': '10px auto'}
    ),
    dcc.Graph(id='stock-graph')
])

@app.callback(
    Output('stock-graph', 'figure'),
    Input('ticker-dropdown', 'value')
)
def update_graph(selected_ticker):
    df_company = df_earn.reset_index().set_index("date").query("ticker == @selected_ticker")
    price_df = data("market/prices", tickers=[selected_ticker]).reset_index()
    return create_figure(df_company, price_df, selected_ticker)

if __name__ == '__main__':
    app.run_server(debug=True)